<a href="https://colab.research.google.com/github/omkargolatkar/actin_analysis/blob/main/Interface_matrix_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\color{Black}{\LARGE \mbox{Interface Matrix}}$

## Preparation

In [1]:
#@title <font color='royalblue'> Change working directory </font>

your_directory = "\"/content/drive/MyDrive/Actin_analysis\"" # @param {type:"string"}

%cd $your_directory

# from google.colab import drive
# drive.mount('/content/drive')

/content/drive/.shortcut-targets-by-id/1ZAWDV03m1MDB61agRZJLx00Fz-zr8uQq/Actin_analysis


In [ ]:
#@title <font color='royalblue'> Install and import required modules </font>

%%capture
!pip install biopython
!pip install python-graphql-client
!pip install py3Dmol
!pip install -U prody

import pandas as pd
import os
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio import SeqIO
import requests, sys, json
import collections
import itertools
import time
from pprint import pprint
from python_graphql_client import GraphqlClient
from collections import Counter
from IPython.display import HTML, display
import time
from google.colab import data_table
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import py3Dmol
import prody

bold_start = "\033[1m"
bold_end = "\033[0m"

In [ ]:
#@title <font color='royalblue'>functions</font>
#@markdown - **residue_label_seq_ids** is according to PDB numbering
#@markdown - **residue_seq_ids** is according to author numbering

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

def get_assembly_info(pdb_id,api_url):
  # Make a GET call to the API URL
    get_request = requests.get(url=api_url+"/"+pdb_id)
    if get_request.status_code == 200:
        # If there is data returned (with HTML status code 200)
        # then return the data in JSON format
        return get_request.json()
    else:
        # If there is no data, print status code and response
        print(get_request.status_code, get_request.text, pdb_id)
        return None

def get_interface_assembly_info(pdb_id,assembly_id,api_url):
  # Make a GET call to the API URL
    get_request = requests.get(url=api_url+"/"+pdb_id+"/"+assembly_id)
    if get_request.status_code == 200:
        # If there is data returned (with HTML status code 200)
        # then return the data in JSON format
        # print(pdb_id + "_"+assembly_id, " ok")
        return get_request.json()
    else:
        # If there is no data, print status code and response
        print(f"{pdb_id}_{assembly_id}")
        return None

# residue_numbering_type = "residue_label_seq_ids" #@param ['residue_label_seq_ids','residue_seq_ids'] {type:"string"}

def parse_interface_data_2(assembly, interface_data, chain_dict, residue_numbering_type = 'residue_label_seq_ids'):
  interface_dict = {}
  partner_interface_dict = {}
  pdb_id = assembly.partition("_")[0]
  assembly_id = assembly.partition("_")[2]

  for interface in interface_data[pdb_id.lower()]['assembly']['interfaces']:
    # interacting chains

    if interface['molecules'][0]['molecule_class'] == 'Protein' and interface['molecules'][1]['molecule_class'] == 'Protein':
      partner1 = interface['molecules'][0]
      partner2 = interface['molecules'][1]
      p1_residues = partner1[residue_numbering_type] # residue_label_seq_ids (pdb sequence numbering) or residue_seq_ids (auth sequence numbering)
      p2_residues = partner2[residue_numbering_type]
      p1_interface_res = []
      for idx in range(0,len(p1_residues)):
        if partner1['buried_surface_areas'][idx] != 0:
          p1_interface_res.append(int(p1_residues[idx]))
      #print(p1_interface_res)
      p2_interface_res = []
      for idx in range(0,len(p2_residues)):
        if partner2['buried_surface_areas'][idx] != 0:
          p2_interface_res.append(int(p2_residues[idx]))
      p1_chain = partner1['chain_id']
      p2_chain = partner2['chain_id']

      chain_lst = chain_dict[pdb_id].split(",")
      #print(chain_lst)

      if p1_chain in chain_lst and p2_chain not in chain_lst:
        #key = (f"{pdb_id}"f"_{assembly_id}-({p1_chain}_act)-{p2_chain}")
        key = (pdb_id + "_" + assembly_id, p1_chain+"_act", p2_chain)
        interface_dict[key] = p1_interface_res
        partner_interface_dict[key] = p2_interface_res
      elif p2_chain in chain_lst and p1_chain not in chain_lst:
        #key = (f"{pdb_id}"f"_{assembly_id}-({p2_chain}_act)-{p1_chain}")
        key = (pdb_id + "_" + assembly_id, p2_chain+"_act", p1_chain)
        interface_dict[key] = p2_interface_res
        partner_interface_dict[key] = p1_interface_res
      elif p1_chain in chain_lst and p2_chain in chain_lst:
        #key = (f"{pdb_id}"f"_{assembly_id}-({p1_chain}_act)-({p2_chain}_act)")
        key = (pdb_id + "_" + assembly_id, p1_chain+"_act", p2_chain+"_act")
        interface_dict[key] = p1_interface_res
        partner_interface_dict[key] = p2_interface_res
        #key = (f"{pdb_id}"f"_{assembly_id}-({p2_chain}_act)-({p1_chain}_act)")
        key = (pdb_id + "_" + assembly_id, p2_chain+"_act", p1_chain+"_act")
        interface_dict[key] = p2_interface_res
        partner_interface_dict[key] = p1_interface_res

  return interface_dict, partner_interface_dict

def get_sifts_uni_pdb_map(pdb_id,api_url):
  # Make a GET call to the API URL
    get_request = requests.get(url=api_url+"/"+pdb_id)
    if get_request.status_code == 200:
        # If there is data returned (with HTML status code 200)
        # then return the data in JSON format
        print(pdb_id, " ok")
        return get_request.json()
    else:
        # If there is no data, print status code and response
        print(get_request.status_code, pdb_id)
        return None

In [ ]:
#@title API links

assembly_info_url = "https://www.ebi.ac.uk/pdbe/api/pdb/entry/assembly" #@param {type:"string"}
#markdown - **api_url** `:` https://www.ebi.ac.uk/pdbe/api/pdb/entry/assembly/2oan

interface_info_url = "https://www.ebi.ac.uk/pdbe/api/pisa/interfaces" #@param {type: "string"}
#markdown - **api_url** `:` https://www.ebi.ac.uk/pdbe/api/pisa/interfaces/2oan/1


## Actin dataset

In [ ]:
#@title Actin dataset
#@markdown Load the actin structure dataset sheet (ouptut of [Actin_dataset_generation.ipynb](https://colab.research.google.com/drive/1DyAz6GTxLoOZFs9TfFuzMJu7OmNbyR3a?usp=sharing))
#@markdown Apply resolution cutoff (in Å)
df = pd.read_excel("./actin_dataset/actin_database_sheets/actin_all_entities_NA.xlsx")

df = df.iloc[:,1:]
# df3 = df3.iloc[:,1:]

df = df[df.resolution != 'none']
df.reset_index(drop=True)
df.index+=1
# df3.index+=1

filament_resolution_cutoff = 4 #@param {type: "number"}
non_filament_resloution_cutoff = 3.5 #@param {type: "number"}

# resolution cutoff of 4 A on filament and 3.5 on non-filament
df_data = df.query('(filament_status == "filament" & resolution <={fil_res}) | (filament_status == "non_filament" & resolution <={nonfil_res})'.format(fil_res = filament_resolution_cutoff, nonfil_res = non_filament_resloution_cutoff))
actin_uniprots = list(set(list(df_data['uniprot_id'])))

#@markdown Outputs -
#@markdown - **actin_pdbs** `:` pdb ids in the selected dataset **df_data**
actin_pdbs = list([entity[0:4] for entity in list(df_data['entity_id'])])

print("There are " + bold_start+ "{num_str}".format(num_str = len(list(set(actin_pdbs)))) + bold_end + " rows in the dataset after the resolution cutoff: ","\n")
print("More than one actin entities in the structures: \n",','.join(list(set([x for x in actin_pdbs if actin_pdbs.count(x) > 1]))))

actin_pdbs = list(set([entity[0:4] for entity in list(df_data['entity_id'])]))
print("actin_pdbs ({num_pdbs}): ".format(num_pdbs = len(actin_pdbs)), actin_pdbs)

# get uniprot ids of actin entities
#@markdown - **actin_uniprots** `:` uniprot ids of actin entities in **df_data**
actin_uniprots = list(set(list(df_data['uniprot_id'])))

#@markdown - **pdb_sequences** `:` dictionary with - {**PDB_id : PDB sequence of actin entity**}
pdb_sequences = {}
for index, row in df_data.iterrows():
  pdb_sequences[row['entity_id'][0:4]] = row['pdb_sequence']

df_data.reset_index(inplace = True, drop = True)
df_data.index+= 1
data_table.DataTable(df_data, num_rows_per_page = 5)

## Assembly information


In [ ]:
#@title Download assembly information
#@markdown For each pdb, the file contains information about entities in each assembly with their chain ids, names and copy number

savepath = "./actin_dataset/assemblies"
os.makedirs(os.path.dirname(savepath+"/tempfile"), exist_ok=True)

#@markdown - **actin_pdbs** `:` non-redundant list of all pdb ids in the **df_data**
actin_pdbs = list(set([entity[0:4] for entity in list(df_data['entity_id'])])) #list of all actin pdbs in selected dataset

out = display(progress(0, 100), display_id=True)
bar_val = 0
for pdb_id in actin_pdbs:
    completeName = os.path.join(savepath,str(pdb_id)+'.json')
    if os.path.exists(completeName) == False:
      assembly_info = get_assembly_info(pdb_id,assembly_info_url)
      # Serializing json
      json_object = json.dumps(assembly_info, indent=4)

      # Writing to sample.json
      with open(completeName, "w") as outfile:
          outfile.write(json_object)
    #print("done " + pdb_id)
    bar_val += 1
    out.update(progress(bar_val,len(actin_pdbs)))

In [ ]:
#@title Save assembly information to dataframe (do not run everytime, 3 min)
# do not run every time, take data from saved file

master_lst = []
for pdb_id in actin_pdbs:
    completeName = "./actin_dataset/assemblies/"+pdb_id+".json"
    with open(completeName, 'r') as openfile:
        result = json.load(openfile)
        for assembly in result[pdb_id.lower()]:
          polymeric_count = assembly['polymeric_count']
          details = assembly['details']
          assembly_id = assembly['assembly_id']
          master_lst.append([pdb_id,polymeric_count, details, assembly_id])

#@markdown - **assembly_df** `:` dataframe with information about type of assembly (author or software defined)
assembly_df = pd.DataFrame(master_lst, columns = ['pdb_id', 'poly_count','details','assembly_id'])
assembly_df.to_excel("./actin_dataset/actin_database_sheets/assembly_info.xlsx")

In [ ]:
#@title Load assembly_df dataframe
assembly_df = pd.read_excel("./actin_dataset/actin_database_sheets/assembly_info.xlsx")
assembly_df = assembly_df.iloc[:,1:]
assembly_df.index+=1
data_table.DataTable(assembly_df, num_rows_per_page = 5)

## Interface information

In [ ]:
#@title Interfaces in each assembly in the **assembly_df**
savepath = "./actin_dataset/interface_assemblies"
os.makedirs(os.path.dirname(savepath+"/tempfile"), exist_ok=True)

#@markdown - **actin_assemblies** `:` actin containing assembly ids list
actin_assemblies = []
for index, row in assembly_df.iterrows():
    pdb_assem = row['pdb_id']+"_"+str(row["assembly_id"])
    actin_assemblies.append(pdb_assem)

print("No interface information found for -")
out = display(progress(0, 100), display_id=True)
bar_val = 0
for assembly in actin_assemblies:
    pdb_id = assembly.partition("_")[0].lower()
    assembly_id = assembly.partition("_")[2]
    completeName = os.path.join(savepath,str(assembly)+'.json')
    if os.path.exists(completeName) == False or os.path.getsize(completeName) < 10:
      assembly_interface_info = get_interface_assembly_info(pdb_id,assembly_id,interface_info_url)
      # Serializing json
      json_object = json.dumps(assembly_interface_info, indent=4)

      # Writing to sample.json
      with open(completeName, "w") as outfile:
          outfile.write(json_object)
    bar_val += 1
    out.update(progress(bar_val,len(actin_pdbs)))
    #print("done " + assembly)

## interface dictionary
- this dictionary is unmodified dictionary of interface residues

In [ ]:
#@title Interface dictionary
interface_assembly_dir = "./actin_dataset/interface_assemblies" #param {type: "string"}
assembly_dir = "./actin_dataset/assemblies" #param {type: "string"}

#@markdown some lists are not exactly duplicates but more or less similar, so delete based on how similar the interfaces are
#@markdown change the **similarity threshold** above to change the fraction of similarity

#@markdown - **all_actin_entities** `:` list of actin entity ids in **df_data**
all_actin_entities = list(df_data['entity_id'])
#@markdown - **chain_dict** `:` dictionary with - {**pdb id** : **actin chains**}
chain_dict = {} # dictionary with key: pdb_id and value: actin chain_ids
#@markdown - **uni_map** `:` dictionary with - {**pdb id** : **actin uniprot id**}
uni_map ={} # dictionary with key: pdb_id and value: uniprot id of actin entity


for index, row in df_data.iterrows():
  chain_dict[row['entity_id'][0:4]] = ''
  uni_map[row['entity_id'][0:4]] = row['uniprot_id']
for index, row in df_data.iterrows():
  chain_dict[row['entity_id'][0:4]] = (','.join([chain_dict[row['entity_id'][0:4]],row['auth_chain_id']]))
for key, value in chain_dict.items():
  chain_dict[key] = value[1:]

#@markdown - **interface_assemblies_to_look** `:` list of non-empty interface files (empty interface file may indicate lack of protein-protein interface)
# look at non-empty interface assembly files
interface_assemblies_to_look = []
for interface_assembly in os.listdir(interface_assembly_dir)[0:]:
  interface_file_path = os.path.join(interface_assembly_dir,interface_assembly)
  filesize = os.path.getsize(interface_file_path)
  if filesize >= 10 and interface_assembly.partition(".")[0] in actin_assemblies:
    interface_assemblies_to_look.append(interface_assembly)

#@markdown - **all_actin_interfaces_dict** `:` dictionary with - {**assembly_id-chain1-chain2 : list of interface residues of chain1** (pdb sequence numbering)}
all_actin_interfaces_dict = {}
all_actin_interfaces_chimerax_dict = {}
all_partner_interfaces_dict = {}
all_partner_interfaces_chimerax_dict = {}

for interface_assembly in interface_assemblies_to_look[0:]:
  interface_file_path = os.path.join(interface_assembly_dir,interface_assembly)
  assembly = interface_assembly.partition(".")[0]
  pdb_id = assembly.partition("_")[0]
  f = open(interface_file_path)
  interface_assembly_data = json.load(f)
  f.close()
  if interface_assembly_data[pdb_id.lower()]['assembly']['interface_count'] != 0:
    interface_file, partner_interface_file = parse_interface_data_2(assembly,interface_assembly_data,chain_dict) # dictionary with key: assembly_id-(chain1)-(chain2) and value: interface residues of chain1
    interface_file_chimerax, partner_interface_file_chimerax = parse_interface_data_2(assembly,interface_assembly_data,chain_dict,residue_numbering_type = 'residue_seq_ids') # dictionary with key: assembly_id-(chain1)-(chain2) and value: interface residues of chain1
  for key, value in interface_file.items():
    all_actin_interfaces_dict[key] = value
    all_actin_interfaces_chimerax_dict[key] = interface_file_chimerax[key]
  for key, value in partner_interface_file.items():
    all_partner_interfaces_dict[key] = value
    all_partner_interfaces_chimerax_dict[key] = partner_interface_file_chimerax[key]

In [ ]:
#@title check interface dictionary

pdb_id = "4CBW" #@param {type:"string"}
for key, value in all_actin_interfaces_dict.items():
  if pdb_id in key[0]:
    print(bold_start+"Interface residues of actin (PDB numbering):"+bold_end, key, value)
    print(bold_start+"Interface residues of actin (author numbering):"+bold_end, key, all_actin_interfaces_chimerax_dict[key])
    # print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict[key])

Interface residues of actin (PDB numbering): ('4CBW_1', 'A_act', 'G') [25, 26, 27, 145, 146, 147, 148, 149, 150, 151, 168, 169, 170, 171, 298, 330, 336, 343, 344, 346, 347, 348, 350, 351, 352, 353, 354, 356, 357]
Interface residues of actin (author numbering): ('4CBW_1', 'A_act', 'G') [24, 25, 26, 144, 145, 146, 147, 148, 149, 150, 167, 168, 169, 170, 297, 329, 335, 342, 343, 345, 346, 347, 349, 350, 351, 352, 353, 355, 356]


In [ ]:
#@title get interface residues to visualise on structure

assembly_pdb_ids = []
chain_ids_dict = defaultdict(list)
res_ids_dict = defaultdict(list)
pdb_id = "4CBW" #@param {type:"string"}
for key, value in all_actin_interfaces_chimerax_dict.items():
  if pdb_id in key[0]:
    # print(bold_start+"Interface residues of actin (author numbering):"+bold_end)
    # print("chain ",key[1], ','.join([str(x) for x in value]))
    # print("chain ", key[2], ','.join([str(x) for x in all_partner_interfaces_chimerax_dict[key]]))
    assembly_pdb_ids.append((key[0].partition("_")[2], key[1], key[2]))
    chain_ids_dict[key[0].partition("_")[2]].append([key[1].partition("_")[0], key[2].partition("_")[0].partition("-")[0]])
    res_ids_dict[(key[0].partition("_")[2], key[1], key[2])] = [value, all_partner_interfaces_chimerax_dict[key]]
    # print(bold_start+"Interface residues of actin (author numbering):"+bold_end, key, all_actin_interfaces_chimerax_dict[key])
    # print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict[key])

import ipywidgets as widgets
from IPython.display import display
import numpy as np

Dropdown_ = widgets.Dropdown(
    options=assembly_pdb_ids,
    description='Assembly ID',
)
output = widgets.Output()
display(Dropdown_)

Dropdown(description='Assembly ID', options=(('1', 'A_act', 'G'),), value=('1', 'A_act', 'G'))

In [ ]:
#@title load the structure
%%capture
assem_id = Dropdown_.value[0]
chains = [Dropdown_.value[1].partition("_")[0], Dropdown_.value[2].partition("_")[0].partition("-")[0]]
res_ids = res_ids_dict[Dropdown_.value]
assembly_pdb_url = f'https://www.rcsb.org/pdb/files/{pdb_id}.pdb{assem_id}.gz'
assembly_pdb_path = f"./{pdb_id}.pdb{assem_id}"
if os.path.exists(assembly_pdb_path) == True:
  os.remove(f"./{pdb_id}.pdb{assem_id}")
!wget {assembly_pdb_url}
!gunzip '{pdb_id}.pdb{assem_id}.gz'

view = py3Dmol.view()
model = view.getModel()
view.addModel(open(f'{pdb_id}.pdb{assem_id}', 'r').read(), 'pdb')
os.remove(f"./{pdb_id}.pdb{assem_id}")

In [ ]:
#@title view the interface
resid_hover = """
function(atom,viewer) {
    if(!atom.label) {
        atom.label = viewer.addLabel(atom.chain+" "+atom.resn+" "+atom.resi,
            {position: atom, backgroundColor: 'mintcream', fontColor:'black', fontSize:12});
    }
}"""
unhover_func = """
function(atom,viewer) {
    if(atom.label) {
        viewer.removeLabel(atom.label);
        delete atom.label;
    }
}"""
representation = 'cartoon'

view.setBackgroundColor('white')
view.setStyle({'chain':chains[0]}, {'cartoon':{'color':'violet'}})
view.setStyle({'chain':chains[0],'resi':res_ids[0]},{representation:{'color':"red"}});
view.setStyle({'chain':chains[1]}, {'cartoon':{'color':'cornflowerblue'}})
view.setStyle({'chain':chains[1],'resi':res_ids[0]},{representation:{'color':"darkblue"}});
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'darkblue'}, {'chain':chains[1],'resi':res_ids[1]})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'red'}, {'chain':chains[0],'resi':res_ids[0]})

model.setHoverable({'chain':chains[1],'resi':res_ids[1]}, True, resid_hover, unhover_func)
model.setHoverable({'chain':chains[0],'resi':res_ids[0]}, True, resid_hover, unhover_func)

view.zoomTo({'chain':chains})
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Remove software defined assemblies (if no. of assemblies > 1)
assembly_info_df = pd.read_excel("./actin_dataset/actin_database_sheets/assembly_info.xlsx")
assembly_info_df = assembly_info_df.iloc[:,1:]
assembly_info_df = assembly_info_df[assembly_info_df.assembly_id != 1]
assembly_info_df.reset_index()
assembly_info_df.index+= 1

remove_rows_lst = []
for index, row in assembly_info_df.iterrows():
  if row['details'] == 'software_defined_assembly':
    remove_rows_lst.append(row['pdb_id']+"_"+str(row["assembly_id"]))

rem_dict = {}

for item in remove_rows_lst:
  for key, value in all_actin_interfaces_dict.items():
    if item  in key:
      #print(key, item)
      rem_dict[key] = value

all_actin_interfaces_dict = {k: v for k, v in all_actin_interfaces_dict.items() if k not in rem_dict}

rem_dict = {}

for item in remove_rows_lst:
  for key, value in all_partner_interfaces_dict.items():
    if item  in key:
      #print(key, item)
      rem_dict[key] = value

all_partner_interfaces_dict = {k: v for k, v in all_partner_interfaces_dict.items() if k not in rem_dict}

In [ ]:
#@title check interface dictionary

pdb_id = "4Z94" #@param {type:"string"}
for key, value in all_actin_interfaces_dict.items():
  if pdb_id in key[0]:
    print(bold_start+"Interface residues of actin (PDB numbering):"+bold_end, key, value)
    print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict[key])

Interface residues of actin (PDB numbering): ('4Z94_1', 'A_act', 'G') [25, 26, 27, 39, 40, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 59, 62, 63, 66, 68, 70, 74, 79, 80, 81, 82, 83, 85, 86, 114, 116, 117, 119, 120, 123, 124, 127, 128, 145, 146, 147, 148, 149, 150, 151, 168, 169, 170, 171, 298, 336, 343, 344, 346, 347, 348, 350, 351, 352, 353, 354, 356, 357, 361, 363, 364, 365, 366, 367, 368, 369, 374, 375]
Interface residues of interactor (PDB numbering): ('4Z94_1', 'A_act', 'G') [1, 2, 3, 4, 5, 8, 25, 26, 63, 65, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 86, 90, 92, 93, 94, 96, 124, 125, 126, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 176, 178, 179, 180, 181, 207, 208, 233, 235, 236, 237, 238, 261, 263, 264, 265, 266, 288, 289, 290, 291, 292, 293, 294, 295, 300, 301, 304, 305, 307, 308, 311, 312, 314, 315, 318, 319, 322]


In [ ]:
for k, v in all_actin_interfaces_dict.items():
  if 5<= len(v) <= 10:
    print(k,v)

## pdb data api

In [ ]:
#@title Actin companions in the dataset (using PDB-data-API)
ids_lst = json.dumps(actin_pdbs) #string of list of entry ids from search api

#data api to get structure title
query = '''
{
  entries(entry_ids: %s) {
    rcsb_id
    polymer_entities {
      polymer_entity_instances {
        rcsb_polymer_entity_instance_container_identifiers {
          asym_id
          auth_asym_id
        }
      }
      rcsb_polymer_entity {
        pdbx_description
      }
      rcsb_polymer_entity_container_identifiers {
        entity_id
        reference_sequence_identifiers {
          database_accession
          database_name
        }
      }
    }
  }
}
'''

data_query = query % (ids_lst)
url_data_api = 'https://data.rcsb.org/graphql' #param {type:"string"}
# instantiate client with the RCSB Data API endpoint
client = GraphqlClient(endpoint = url_data_api)
#pdb_string = ','.join(map("'{0}'".format, pdb_ids))
#query_variables = { 'entry_ids': pdb_string }
result = client.execute(query=data_query)
result = result['data']
#pprint(result['entries'][0:1])

#save output
# Serializing json
json_object = json.dumps(result, indent=4)

In [ ]:
#@title Actin and its companions in the dataset (JSON to Dataframe)

master_lst = []
for entry in result['entries']:
  pdb_id = entry['rcsb_id']

  for entity in entry['polymer_entities']:
    try:
      accessions = []
      for acc in entity['rcsb_polymer_entity_container_identifiers']['reference_sequence_identifiers']:
        if acc['database_name'] == "UniProt":
          accessions.append(acc['database_accession'])
      uni_acc = ','.join(accessions)
      entity_uniprot_id = uni_acc
    except TypeError:
      entity_uniprot_id = 'none'
      accessions = ['none']

    accessions_mod = []
    if len(accessions) > 1:
      for uni_id in accessions:
        if len(uni_id) <= 6:
          accessions_mod.append(uni_id)
    else:
      accessions_mod = accessions.copy()
    entity_id = entity['rcsb_polymer_entity_container_identifiers']['entity_id']
    #pdb chains
    pdb_chains = []
    auth_chains = []
    for chain_dict in entity['polymer_entity_instances']:
      pdb_chains.append(chain_dict['rcsb_polymer_entity_instance_container_identifiers']['asym_id'])
      auth_chains.append(chain_dict['rcsb_polymer_entity_instance_container_identifiers']['auth_asym_id'])
    asym_ids = ','.join(pdb_chains)
    auth_asym_ids = ','.join(auth_chains)
    entity_name = entity['rcsb_polymer_entity']['pdbx_description']

    for unp in accessions_mod:
      #if unp not in actin_uniprots:
        master_lst.append([pdb_id,entity_id,unp,asym_ids,auth_asym_ids,entity_name])
#@markdown - **df_entities** `:` all entities in all the actin structures
df_entities = pd.DataFrame(master_lst,columns=['pdb_id','entity_id','uniprot_id','pdb_chain_id','auth_chain_id','entity_name'])
df_entities = df_entities[df_entities.uniprot_id != 'none']
df_entities.reset_index(inplace = True, drop = True)
df_entities.index+=1 # all entities in all the actin structures

# df_entities.loc[df_entities['uniprot_id'] == 'L0I5A1', 'uniprot_id'] = 'Q87GE5'
data_table.DataTable(df_entities, num_rows_per_page = 5)

In [ ]:
# @title
# #@title separate actin-actin and actin-protein interfaces
# all_actin_interfaces_dict_act_act = {}
# all_actin_interfaces_dict_act_prot = {}
# all_partner_interfaces_dict_act_act = {}
# all_partner_interfaces_dict_act_prot = {}
# for k, v in all_actin_interfaces_dict.items():
#   if 'act' in k[1] and 'act' in k[2]:
#     all_actin_interfaces_dict_act_act[k] = v
#     all_partner_interfaces_dict_act_act[k] = all_partner_interfaces_dict[k]
#   elif k[1].partition("_")[0] == k[2].partition("-")[0]:
#     all_actin_interfaces_dict_act_act[k] = v
#     all_partner_interfaces_dict_act_act[k] = all_partner_interfaces_dict[k]
#   else:
#     all_actin_interfaces_dict_act_prot[k] = v
#     all_partner_interfaces_dict_act_prot[k] = all_partner_interfaces_dict[k]

In [ ]:
# @title
# #@title remove duplicates
# similarity_threshold = 0.8 #@param {type: "number"}

# all_actin_interfaces_dict_unique = {}
# all_partner_interfaces_dict_unique = {}
# for interface_assembly in interface_assemblies_to_look[0:]:
#   assembly = interface_assembly.partition(".")[0]
#   pdb_id = assembly.partition("_")[0]
#   res1 = {k:v for k,v in all_actin_interfaces_dict.items() if k[0] == assembly}
#   res2 = {k:v for k,v in all_partner_interfaces_dict.items() if k[0] == assembly}

#   for key1, value1 in res1.items():
#     for key2, value2 in res1.items():
#       a = set(value1)
#       b = set(value2)
#       commonset_len = len(a & b)
#       min_len = min(len(a),len(b))
#       common_fract = commonset_len/min_len
#       #interface_corr[key1+"and"+key2] = common_fract
#       if common_fract >= similarity_threshold:
#         resulting_list = sorted(list(a.union(b)))
#         res1[key1] = resulting_list
#         res1[key2] = resulting_list

#   temp1 = {','.join(str(x) for x in val): key for key, val in res1.items()}
#   res1 = {val: [eval(i) for i in key.split(',')] for key, val in temp1.items()}

#   for k, v in res1.items():
#     all_actin_interfaces_dict_unique[k] = v
#     all_partner_interfaces_dict_unique[k] = res2[k]

In [ ]:
# @title
# for k, v in all_actin_interfaces_dict_unique.items():
#   print(k)

In [ ]:
# @title
# #@title check interface dictionary

# pdb_id = "8C4C" #@param {type:"string"}
# for key, value in all_actin_interfaces_dict_unique.items():
#   if pdb_id in key[0]:
#     print(bold_start+"Interface residues of actin (PDB numbering):"+bold_end, key, value)
# for key, value in all_partner_interfaces_dict_unique.items():
#   if pdb_id in key[0]:
#     print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict_unique[key])

## SIFTS mapping for actin companions

In [ ]:
#@title SIFTS mappings for each PDB
# map pdb sequence to uniprot sequence
# based on mapping update interface residues
# do MSA based on uniprot ids
# based on gapped seq update interface residues

#sifts_url = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot"
sifts_url = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot_segments" #@param {type:"string"}
#@markdown - **sifts_url** `:` https://www.ebi.ac.uk/pdbe/api/mappings/uniprot_segments/2oan
savepath = "./actin_dataset/sifts_uni_pdb_mappings" #param {type:"string"}
os.makedirs(os.path.dirname(savepath+'/tempfile'), exist_ok=True)
out = display(progress(0, 100), display_id=True)
bar_val = 0
for pdb_id in actin_pdbs:
  completeName = os.path.join(savepath,str(pdb_id)+'.json')
  if os.path.exists(completeName) == False:
    sifts_map = get_sifts_uni_pdb_map(pdb_id, sifts_url)
    json_object = json.dumps(sifts_map, indent=4)

    with open(completeName, "w") as outfile:
      outfile.write(json_object)
  bar_val += 1
  out.update(progress(bar_val,len(actin_pdbs)))

In [ ]:
#@title Map uniprot numbering to PDB sequence numbering
sifts_map_dir = "./actin_dataset/sifts_uni_pdb_mappings" #param {type:"string"}

sifts_chain_map = {}
for sifts_map in os.listdir(sifts_map_dir):
  uni_pdb_sifts_map = os.path.join(sifts_map_dir,sifts_map)
  pdb_id = sifts_map.partition(".")[0].lower()
  uniprot_id_lst = []

  f = open(uni_pdb_sifts_map)
  uni_pdb_data = json.load(f)
  f.close()

  inner_dict2 = {}
  uniprot_id_lst = list(uni_pdb_data[pdb_id]['UniProt'].keys())
  for uni_id in uniprot_id_lst:
    map_data = uni_pdb_data[pdb_id]['UniProt'][uni_id]['mappings']
    for mapping in map_data:
      chain_id = mapping['chain_id']
      inner_dict2[chain_id] = {}

  for uni_id in uniprot_id_lst:
    map_data = uni_pdb_data[pdb_id]['UniProt'][uni_id]['mappings']
    for mapping in map_data:
      chain_id = mapping['chain_id']
      inner_dict2[chain_id][uni_id] = []

  for uni_id in uniprot_id_lst:
    map_data = uni_pdb_data[pdb_id]['UniProt'][uni_id]['mappings']
    for mapping in map_data:
      inner_dict2[mapping['chain_id']][uni_id].append({'start_pdb_unp':[mapping['start']['residue_number'], mapping['unp_start']], 'end_pdb_unp':[mapping['end']['residue_number'], mapping['unp_end']]})
  sifts_chain_map[pdb_id] = inner_dict2

In [ ]:
sifts_chain_map['8gt4']

In [ ]:
#@title Update keys of interface residue dictionaries

#markdown - **all_interfaces_dict_with_uniprot** `:` dictionary with - {**(assembly_id, (chain1,ch1_uniprot), (chain2,ch2_uniprot)) : list of interface residues of chain1** (pdb sequence numbering)}
all_actin_interfaces_dict_with_uniprot = {}
all_partner_interfaces_dict_with_uniprot = {}
for key, value in all_actin_interfaces_dict.items():
  pdb_id = key[0].partition("_")[0]
  chain1 = key[1].partition("_")[0]
  chain2 = key[2]
  if "_act" in chain2:
    chain2 = chain2.partition("_")[0]
  elif "-" in chain2:
    chain2 = chain2.partition("-")[0]

  chain1_overlaps = {}
  for uni_id in list(sifts_chain_map[pdb_id.lower()][chain1].keys()):
    ch1_overlap_length = 0
    for fragment in sifts_chain_map[pdb_id.lower()][chain1][uni_id]:
      l = fragment['start_pdb_unp'][0]
      r = fragment['end_pdb_unp'][0]
      ch1_overlap_length+=len(list(x for x in value if l <= x <= r))
    chain1_overlaps[uni_id] = ch1_overlap_length/len(value)
  ch1_unp = max(chain1_overlaps, key= lambda x: chain1_overlaps[x])
  if len(list(chain1_overlaps.keys()))>1:
    print(pdb_id, chain1)
  chain2_overlaps = {}

  try:
    for uni_id in list(sifts_chain_map[pdb_id.lower()][chain2].keys()):
      ch2_overlap_length = 0
      for fragment in sifts_chain_map[pdb_id.lower()][chain2][uni_id]:
        l = fragment['start_pdb_unp'][0]
        r = fragment['end_pdb_unp'][0]
        ch2_overlap_length+= len(list(x for x in all_partner_interfaces_dict[key] if l <= x <= r))
      chain2_overlaps[uni_id] = ch2_overlap_length/len(all_partner_interfaces_dict[key])
      ch2_unp = max(chain2_overlaps, key= lambda x: chain2_overlaps[x])
  except:
    pass
    ch2_unp = 'none'

  for k, v in chain2_overlaps.items():
    if v < 1 and pdb_id == '4Z94':
      print(k, v)
  # if len(list(chain2_overlaps.keys()))>1:
  #   print(pdb_id, chain2)
  newkey = (key[0], (key[1], ch1_unp), (key[2], ch2_unp))
  all_actin_interfaces_dict_with_uniprot[newkey] = value
  all_partner_interfaces_dict_with_uniprot[newkey] = all_partner_interfaces_dict[key]

P06396 0.379746835443038
P28289 0.189873417721519
P29536 0.4177215189873418
4CBW A


In [ ]:
#@title check interface dictionary

pdb_id = "8GT4" #@param {type:"string"}
for key, value in all_actin_interfaces_dict_with_uniprot.items():
  if pdb_id in key[0]:
    print(bold_start+"Interface residues of actin (PDB numbering):"+bold_end, key, value)
    # print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict_with_uniprot[key])

In [ ]:
#@title Map uniprot numbering to PDB sequence numbering
sifts_map_dir = "./actin_dataset/sifts_uni_pdb_mappings" #param {type:"string"}

#@markdown - **actin_uniprots** `:` non-redundant list of all the actin entities in **df_data**
actin_uniprots = list(set(list(uni_map.values())))

#@markdown - **uni_to_pdb_mappings** `:` uniprot to PDB mapping offset for each chain
uni_to_pdb_mappings = {}

for sifts_map in os.listdir(sifts_map_dir):
  uni_pdb_sifts_map = os.path.join(sifts_map_dir,sifts_map)
  pdb_id = sifts_map.partition(".")[0].lower()
  uniprot_id_lst = []

  f = open(uni_pdb_sifts_map)
  uni_pdb_data = json.load(f)
  f.close()
  for unp in list(uni_pdb_data[pdb_id]['UniProt']):
      uniprot_id_lst.append(unp)
  inner_dict2 = {}
  for uniprot_id in uniprot_id_lst:

    inner_dict1 = defaultdict(list)
    for instance in uni_pdb_data[pdb_id]['UniProt'][uniprot_id]['mappings']:
      pdb_start = instance['start']['residue_number']
      pdb_end = instance['end']['residue_number']
      uni_start = instance['unp_start']
      uni_end = instance['unp_end']
      chain_id = instance['chain_id']
      inner_dict1[chain_id].append([pdb_start,pdb_start - uni_start])

      if (pdb_end-pdb_start) != (uni_end-uni_start):
        print(uniprot_id,pdb_id, chain_id, pdb_end, pdb_start, uni_end, uni_start)
    inner_dict2[uniprot_id] = inner_dict1
  uni_to_pdb_mappings[pdb_id] = inner_dict2

In [ ]:
#@title Modified interface dictionary according to uniprot numbering

#@markdown - **all_interfaces_dict_new** `:`
all_interfaces_dict_new = {} #interface residues according to uniprot numbering
for key, value in all_actin_interfaces_dict_with_uniprot.items():
  pdb_id = key[0].partition("_")[0].lower()
  unp_id = key[1][1]
  ch_id = key[1][0].partition("_")[0]
  mapping_offset = uni_to_pdb_mappings[pdb_id][unp_id][ch_id]
  num_lst = len(mapping_offset)
  mod_value = []
  if num_lst > 1:
    for idx in range(0,num_lst-1):
      for interface_res in value:
        if mapping_offset[idx][0] <= interface_res < mapping_offset[idx+1][0]:
          interface_res_mod = interface_res - mapping_offset[idx][1]
          mod_value.append(interface_res_mod)
        elif interface_res >= mapping_offset[idx+1][0]:
          interface_res_mod = interface_res - mapping_offset[idx+1][1]
          mod_value.append(interface_res_mod)
    all_interfaces_dict_new[key] = mod_value
  else:
    for interface_res in value:
      interface_res_mod = interface_res - mapping_offset[0][1]
      mod_value.append(interface_res_mod)
    all_interfaces_dict_new[key] = mod_value

#@markdown - **all_partner_interfaces_dict_new** `:`
all_partner_interfaces_dict_new = {} #interface residues according to uniprot numbering
for key, value in all_partner_interfaces_dict_with_uniprot.items():
  pdb_id = key[0].partition("_")[0].lower()
  unp_id = key[2][1]
  ch_id = key[2][0].partition("_")[0]
  if '-' in ch_id:
    ch_id = ch_id.partition("-")[0]
  mod_value = []
  try:
    mapping_offset = uni_to_pdb_mappings[pdb_id][unp_id][ch_id]
    num_lst = len(mapping_offset)
    if num_lst > 1:
      for idx in range(0,num_lst-1):
        for interface_res in value:
          if mapping_offset[idx][0] <= interface_res < mapping_offset[idx+1][0]:
            interface_res_mod = interface_res - mapping_offset[idx][1]
            mod_value.append(interface_res_mod)
          elif interface_res >= mapping_offset[idx+1][0]:
            interface_res_mod = interface_res - mapping_offset[idx+1][1]
            mod_value.append(interface_res_mod)
      all_partner_interfaces_dict_new[key] = mod_value
    else:
      for interface_res in value:
        interface_res_mod = interface_res - mapping_offset[0][1]
        mod_value.append(interface_res_mod)
      all_partner_interfaces_dict_new[key] = mod_value
  except:
    all_partner_interfaces_dict_new[key] = mod_value

In [ ]:
#@title check interface dictionary

pdb_id = "8GT4" #@param {type:"string"}
for key, value in all_interfaces_dict_new.items():
  if pdb_id in key[0]:
    print(bold_start+"Interface residues of actin (PDB numbering):"+bold_end, key, value)
    # print(bold_start+"Interface residues of interactor (PDB numbering):"+bold_end, key, all_partner_interfaces_dict_with_uniprot[key])

Interface residues of actin (PDB numbering): ('8GT4_1', ('A_act', 'P68032'), ('B', 'Q94707')) [25, 26, 27, 28, 29, 30, 75, 77, 109, 110, 111, 112, 113, 114, 115, 118, 135, 138, 145, 146, 147, 148, 149, 150, 151, 168, 169, 170, 171, 172, 174, 175, 177, 179, 180, 181, 298, 330, 334, 336, 340, 343, 346, 347, 348, 350, 351, 352, 353, 354, 356, 357, 373, 374, 375, 376, 377]


In [ ]:
#@title Save partner interface dictionary with uniprot

savepath = "./actin_dataset/actin_database_sheets"

#json_object = pickle.dumps(all_partner_interfaces_dict_with_uniprot, indent=4)

with open(os.path.join(savepath,"partner_interface_dictionary_unp.pkl"), 'wb') as f:
    # pickle.dump(all_partner_interfaces_dict_with_uniprot, f)
    pickle.dump(all_partner_interfaces_dict_new, f)

## Multiple Sequence Alignment (MSA)

In [ ]:
#@title function
# Documentation: https://www.uniprot.org/help/api
WEBSITE_API = "https://rest.uniprot.org/"

# Documentation: https://www.ebi.ac.uk/proteins/api/doc/
PROTEINS_API = "https://www.ebi.ac.uk/proteins/api"

# Helper function to download data
def get_url(url, **kwargs):
  response = requests.get(url, **kwargs);

  if not response.ok:
    print(response.text)
    response.raise_for_status()
    sys.exit()

  return response

In [ ]:
#@title ClustalO API - submit job
joined = ",".join(actin_uniprots) #comma separated list of actin uniprot ids in string format

r_sequences = get_url(f"{WEBSITE_API}/uniprotkb/accessions?accessions={joined}&format=fasta")
fasta = r_sequences.text
#print(fasta)

your_email = "omkar.golatkar@students.iiserpune.ac.in" #@param {type:"string"}
output_order = "aligned" # @param ["aligned", "input"]
output_format = "clustal_num" #@param ["clustal_num","clustal","fa","phylip"]
# submit align job using clustalo
req_msa = requests.post("https://www.ebi.ac.uk/Tools/services/rest/clustalo/run", data={
    "email": your_email,
    "iterations": 0,
    "outfmt": output_format,
    "order": output_order,
    "sequence": fasta #fasta or input
})
# documentation here https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/Clustal+Omega+Help+and+Documentation#ClustalOmegaHelpandDocumentation-RESTAPI

job_id_msa = req_msa.text
print(job_id_msa)

<?xml version='1.0' encoding='UTF-8'?>
<error>
 <description>Invalid parameters: 
Sequence -> Error in reading input sequence. Please check your input.</description>
</error>


In [ ]:
#@title ClustalO API - get job status
# # # get job status
r_msa_status = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id_msa}")
print(r_msa_status.text)

FINISHED


In [ ]:
#@title ClustalO API - save MSA
r_msa_clustal = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id_msa}/aln-clustal_num")
#print(r.text)
savepath = "./actin_dataset" #param {type:"string"}

MSA_filename = "actin_MSA.clustal_num" #@param {type:"string"}
with open(os.path.join(savepath, MSA_filename), "w") as f:
  f.write(r_msa_clustal.text)

sequence_filename = "actin_sequences.fasta" #@param {type:"string"}
with open(os.path.join(savepath, sequence_filename),"w") as g:
  g.write(fasta)

In [ ]:
#@title Consensus sequence
alignments = AlignIO.read("./actin_dataset/actin_MSA.clustal_num", "clustal")
#print(dir(alignment))

#@markdown - **align_dict** `:` dictionary with - {**uniprot_id : aligned sequence**}
align_dict = {}
for seq_rec in alignments._records:
  uniprot_id = (seq_rec.id).partition("|")[2].partition("|")[0]
  align_dict[uniprot_id] = seq_rec.seq
#print(align_dict)
summary_align = AlignInfo.SummaryInfo(alignments)

import warnings
warnings.filterwarnings('ignore')
#@markdown - **gap_con** `:` consensus sequence of MSA
gap_con = summary_align.gap_consensus(1,"X")

print(gap_con)

counter = 0 #199
for let in gap_con:
  if let != 'X':
    counter+=1
#################

XXXXXXXXXXVXDNGXGXXKXGXXGDDAPXXVFPSXXGXPXXXXXMXGXXXKXXXVGDEAQXKRGXLXLXYPIEHGXXXNWDDMEKIWXHXFXNXXRXXPEXHXXLLTEAPXNPKXNREXMTQIXFEXFXXPXXXXXIQAXLSLYXSGRTTGXVXDXGDGVXHXVPXYXGXXLPHAXXRXDXAGRDLTXXXMKXXXEXGXXFXTXAEXEIVRXXKEXLXYXAXXXXXEXXXXXXXXXXXEXXXELPDGXXXXXGXXRFRXPEXXFXPXXXGXXXXXGXXXXXXXXIXKCDXDXRXXLYXNXVXSGGXTMXXXXXXRXXXXXXXLAPXXXKXKXXAPPERKYSVWIGGSILXSLXTFQXMWXXKXEYXXXGPXIVHXKCF


In [ ]:
#@title Sequence dictionaries
records = list(SeqIO.parse("./actin_dataset/actin_sequences.fasta", "fasta"))
#@markdown - **uniprot_dict** `:` dictionary with {**actin_uniprot_id : uniprot sequence**}
uniprot_dict = {}
#@markdown - **gapped_dict** `:` dictionary with {**actin_uniprot_id : aligned sequence**}
gapped_dict = {}
for seq_rec in records:
  uniprot_id = (seq_rec.id).partition("|")[2].partition("|")[0]
  uniprot_dict[uniprot_id] = str(seq_rec.seq)

for key, value in align_dict.items():
  gapped_dict[key] = str(value)

In [ ]:
#@title Modified interface dictionary 2

#markdown - **all_interfaces_dict_new_gap** `:` dictionary with - {**assembly_id-chain1-chain2 : list of interface residues of chain1** (aligned sequence numbering)}
all_interfaces_dict_new_gap = {}
for key, value in all_interfaces_dict_new.items():
  pdb_id = key[0].partition("_")[0]
  uniprot_id = uni_map[pdb_id]
  gapped_seq = gapped_dict[uniprot_id]
  uniprot_seq = uniprot_dict[uniprot_id]
  gap_idx = []
  value_lst = value.copy()
  for i in range(0,len(gapped_seq)):
    if gapped_seq[i] == '-':
      gap_idx.append(i+1)
  mod_value_lst = [x for x in value_lst]
  for interface_res in value_lst:
    for item in gap_idx:
      if interface_res >= item:
        mod_value_lst.remove(interface_res)
        interface_res+=1
        mod_value_lst.append(interface_res)

  mod_value_lst = list(set(mod_value_lst))
  mod_value_lst.sort()
  all_interfaces_dict_new_gap[key] = mod_value_lst

In [ ]:
#@title Check all the interface residue dictionaries
pdb_id = "8GT4" #@param {type:"string"}

for key, value in all_interfaces_dict_new_gap.items():
  if pdb_id in key[0]:
    #print(bold_start+"Interface residues (PDB numbering):"+bold_end,key, value)
    print(bold_start+"Interface residues (uniprot numbering):"+bold_end,key, all_interfaces_dict_new[key])
    print(bold_start+"Interface residues (MSA numbering):"+bold_end,key, all_interfaces_dict_new_gap[key])

Interface residues (uniprot numbering): ('8GT4_1', ('A_act', 'P68032'), ('B', 'Q94707')) [25, 26, 27, 28, 29, 30, 75, 77, 109, 110, 111, 112, 113, 114, 115, 118, 135, 138, 145, 146, 147, 148, 149, 150, 151, 168, 169, 170, 171, 172, 174, 175, 177, 179, 180, 181, 298, 330, 334, 336, 340, 343, 346, 347, 348, 350, 351, 352, 353, 354, 356, 357, 373, 374, 375, 376, 377]
Interface residues (MSA numbering): ('8GT4_1', ('A_act', 'P68032'), ('B', 'Q94707')) [25, 26, 27, 28, 29, 30, 75, 77, 109, 110, 111, 112, 113, 114, 115, 118, 135, 138, 145, 146, 147, 148, 149, 150, 151, 168, 169, 170, 171, 172, 174, 175, 177, 179, 180, 181, 300, 332, 336, 338, 342, 345, 348, 349, 350, 352, 353, 354, 355, 356, 358, 359, 375, 376, 377, 378, 379]


In [ ]:
#@title Save interface dictionary

savepath = "./actin_dataset/actin_database_sheets"

with open(os.path.join(savepath,"interface_dictionary_msa.pkl"), 'wb') as f:
    pickle.dump(all_interfaces_dict_new_gap, f)

## Interface Matrix

In [ ]:
#@title Consensus sequence dictionary
#@markdown - **consensus_dict** `:` dictionary of consensus sequence with - {**residue position : residue**}
consensus_dict = {}
for i in range(0,len(gap_con)):
  consensus_dict[i+1] = gap_con[i]

In [ ]:
#@title Interface matrix generation
#@markdown **heatmap_dict** `:` dictionary of every value in interface matrix with - {**assembly_id-(chain1)-(chain2) : 0 or 1**}
heatmap_dict = {}
# for interface, res_lst in all_interfaces_dict_new_gap.items():
#   for res_idx, residue in consensus_dict.items():
#     heatmap_dict[(interface,res_idx)] = 0.0
#   for res in res_lst:
#     heatmap_dict[(interface,res)] = 1.0

for res_idx, residue in consensus_dict.items():
  for interface, res_lst in all_interfaces_dict_new_gap.items():
    heatmap_dict[(interface,res_idx)] = 0.0
  for res in res_lst:
    heatmap_dict[(interface,res)] = 1.0

print("\033[1m" + "Number of rows in interface matrix: " + "\033[0;0m")
print(len(heatmap_dict.keys())/379)

for key, value in heatmap_dict.items():
  if key[1] > 379:
    print(key)

Number of rows in interface matrix: 
2771.0


In [ ]:
#@title dictionary (**heatmap_dict**) to dataframe
ser = pd.Series(list(heatmap_dict.values()),index=pd.MultiIndex.from_tuples(heatmap_dict.keys()))
interface_df = ser.unstack().fillna(0)
print("\033[1m" + "Dimensions of interface matrix: " + "\033[0;0m")
print(interface_df.shape)

interface_matrix_dict = defaultdict(list)
for key, value in heatmap_dict.items():
  newkey = key[0]
  interface_matrix_dict[newkey].append(value)

Dimensions of interface matrix: 
(2771, 379)


In [ ]:
#@title Save interface matrix
savepath = "./actin_dataset/actin_database_sheets" #@param {type: "string"}
interface_df.to_csv(os.path.join(savepath,"actin_interface_matrix.csv"))
#result_tr = df.transpose()
# interface_df.to_csv(os.path.join(savepath, "actin_interface_matrix.tsv"), sep="\t")

with open(os.path.join(savepath,"actin_interface_matrix.pkl"), 'wb') as f:
    pickle.dump(interface_matrix_dict, f)

# Domain annotations

In [ ]:
#@title Actin interactors in each PDB
interface_mtrx = "./actin_dataset/actin_database_sheets/actin_interface_matrix.csv"
#markdown - **mtrx** `-` row names of interface matrix
mtrx = pd.read_csv(interface_mtrx, index_col = 0)

In [ ]:
#@title Actin interactors uniprots

mtrx_rows = list(mtrx.index.values)
mtrx_rows_tuples = []
for elem in mtrx_rows:
  row_elements = elem.split(",")
  assembly_id = row_elements[0].split("'")[1]
  chain_1 = (row_elements[1].partition("'")[2].partition("'")[0], row_elements[2].partition("'")[2].partition("'")[0])
  chain_2 = (row_elements[3].partition("'")[2].partition("'")[0], row_elements[4].partition("'")[2].partition("'")[0])
  mtrx_rows_tuples.append((assembly_id, chain_1, chain_2))

interactors_uniprot_lst = []
for elem in mtrx_rows_tuples:
  actin_interactor_unp = elem[2][1]
  interactors_uniprot_lst.append(actin_interactor_unp)

interactors_uniprot_lst = list(set(interactors_uniprot_lst)) #149 uniprots including actin
only_interactors_uniprot_lst = [elem for elem in interactors_uniprot_lst if elem not in actin_uniprots] #133 from interface matrix
actin_in_interactors_uniprot_lst = [elem for elem in interactors_uniprot_lst if elem in actin_uniprots] #16

In [ ]:
#@title actin structures according to number of interacting partners after curation

act_struct_partner_uniprot_dict = {}
for pdb_id in actin_pdbs:
  act_struct_partner_uniprot_dict[pdb_id] = []
for elem in mtrx_rows_tuples:
  if 'act' not in elem[2][0]:
    actin_interactor_unp = elem[2][1]
    if actin_interactor_unp not in ['Q08641', 'P0CU65', 'P0CU64', 'P0CU63']: # these are lifeact, phalloidin
      PDB_id = elem[0][0:4]
      if actin_interactor_unp not in act_struct_partner_uniprot_dict[PDB_id]:
        act_struct_partner_uniprot_dict[PDB_id].append(actin_interactor_unp)

num_interactors = []
for k, v in act_struct_partner_uniprot_dict.items():
  if len(v) not in num_interactors:
    num_interactors.append(len(v))

num_interactors.sort()

In [ ]:
#@title actin interactors names df
act_struct_partner_names_dict = defaultdict(list)
act_companions_dict = {}

for index, row in df_entities.iterrows():
  act_companions_dict[row['uniprot_id']] = row['entity_name']

for k, v in act_struct_partner_uniprot_dict.items():
  for unip_id in v:
    try:
      act_struct_partner_names_dict[k].append(act_companions_dict[unip_id])
    except KeyError:
      print(unip_id, '-', k)
      pass

master_lst_entries = []
for index, row in df_data.iterrows():
  pdb_id = row['entity_id'][0:4]
  actin_uniprot = row['uniprot_id']
  structure_title = row['structure_title']
  resolution = row['resolution']
  method = row['method']
  filament_status = row['filament_status']
  interactors = ';'.join(act_struct_partner_names_dict[pdb_id])
  master_lst_entries.append([pdb_id,method,resolution,actin_uniprot,filament_status,interactors])

df_entries = pd.DataFrame(master_lst_entries,columns=['PDB ID','Method','Resolution','Actin UniProt ID','Filament status','Actin Interactors'])
df_entries = df_entries.drop_duplicates(subset = ['PDB ID'])
df_entries.index+=1

filename = "./actin_dataset/actin_database_sheets/actin_all_pdbs_res.xlsx"# add a date to the name
os.makedirs(os.path.dirname(filename), exist_ok=True)

#df3.to_excel(os.path.join(savepath,"actin_filament_entities_NA.xlsx"))
df_entries.to_excel(filename)

none - 7ZTC
none - 6C1D
none - 3TPQ
none - 3MN9
none - 7PM5
none - 3MN6
none - 6T1Y
none - 6U96
none - 7PME
none - 8P94
none - 7PMI
none - 7PMF
none - 7P1G
none - 7PML
none - 7PM8
none - 7PLV
none - 8OI6
none - 4M63
none - 3MMV
none - 5NBL
none - 7PDZ
none - 7JH7
none - 6T25
none - 7PMJ
none - 8IAH
none - 5JLH
none - 8IAI
none - 7TPT
none - 7AD9
none - 8A5Q
none - 7PM7
none - 8A5P
none - 5NBM
none - 7PMH
none - 7PMG
none - 7PMD
none - 7PM6
none - 8A5D
none - 6F1T
none - 7PLW
none - 3J8A
none - 6T20
none - 7PLU
none - 7PLT


In [ ]:
bare_count = 0
for index, row in df_entries.iterrows():
  if row['Actin Interactors'] == '' and row['Filament status'] == 'filament':
    bare_count+=1

bare_count

65

In [ ]:
#@title plots
# num_interactors
actin_filament_pdbs = []
for index, row in df_entries.iterrows():
  if row['Filament status'] == 'filament':
    actin_filament_pdbs.append(row['PDB ID'])

rise_twist_pdbs = []
df_rise_twist = pd.read_excel("./actin_dataset/actin_database_sheets/df_rise_twist.xlsx", index_col = 0)
for index, row in df_rise_twist.iterrows():
  if row['prot'] not in ['Bare-Actin','more than one interactors']:
    rise_twist_pdbs.append(index[0:4])

plot_dict = {}
plot_dict2 = {}
for i in range(0,len(num_interactors)):
  counter = 0
  counter_fil = 0
  for k, v in act_struct_partner_uniprot_dict.items():
    if len(v) == num_interactors[i]:
      counter+=1
    if k in actin_filament_pdbs and len(v) == num_interactors[i]:
      counter_fil += 1

  plot_dict[str(num_interactors[i])] = counter - counter_fil
  plot_dict2[str(num_interactors[i])] = counter_fil


x = list(plot_dict.keys())
y1 = list(plot_dict.values())
y2 = list(plot_dict2.values())

fig, ax = plt.subplots()

# bars = ax.bar(x,y1, width = 0.4, label = 'Actin structures')

bars2 = ax.bar(x,y2, width = 0.4, label = 'Structures containing actin filament/s')
#ax.bar_label(bars2)
for c in ax.containers:
    # Filter the labels
    labels = [v if v > 5 else "" for v in c.datavalues]
    ax.bar_label(c, labels=labels, label_type="edge")

bars = ax.bar(x,y1,bottom = y2,color = 'tomato',width= 0.4, label = 'Structures containing non-filamentous actin')
ax.bar_label(bars)

ax.legend()
plt.xlabel('Number of non-actin interactors of actin in the structures')
plt.ylabel('Number of structures deposited in the PDB')
plt.title('Number of actin structures with varying number of interacting partners')

fig.set_size_inches(9, 7)
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)

plt.savefig('./structures_vs_num_interactors_1.png', dpi = 300, bbox_inches = 'tight')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './actin_dataset/actin_database_sheets/df_rise_twist.xlsx'

In [ ]:
#@title functions
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

def get_annot(uniprot_id,api_url):
  # Make a GET call to the API URL
    get_request = requests.get(url=api_url+"/"+uniprot_id)
    if get_request.status_code == 200:
        # If there is data returned (with HTML status code 200)
        # then return the data in JSON format
        return get_request.json()
    else:
        # If there is no data, print status code and response
        print(get_request.status_code, get_request.text, uniprot_id)
        return None

## Representative domains

In [ ]:
#@title download annotation for actin interactors
#P12883 next doesn't work
api_url = "https://www.ebi.ac.uk/interpro/api/entry/all/protein/uniprot"
savepath = "./actin_dataset/interactor_representative_annotations_by_uniprot"
os.makedirs(os.path.dirname(savepath+"/tempfile"), exist_ok=True)
# https://www.ebi.ac.uk/interpro/api/structure/PDB/8bjh/protein/UniProt/
#actin_pdbs = list(set([entity[0:4] for entity in list(df_data['entity_id'])])) #list of all actin pdbs in selected dataset

out = display(progress(0, 100), display_id=True)
bar_val = 0
for uniprot_id in interactors_uniprot_lst:
    completeName = os.path.join(savepath,str(uniprot_id)+'.json')
    if os.path.exists(completeName) == False:
      annot_info = get_annot(uniprot_id,api_url)
      if annot_info != None:
        next = annot_info['next']
        while next != None:
          get_request = requests.get(url=next)
          annot_info1 = get_request.json()
          try:
            for domain in annot_info1['results']:
              annot_info['results'].append(domain)
            next = annot_info1['next']
          except:
            print(uniprot_id)
            next = None
      # Serializing json
      json_object = json.dumps(annot_info, indent=4)

      # Writing to sample.json
      with open(completeName, "w") as outfile:
          outfile.write(json_object)
    #print("done " + uniprot_id)
    time.sleep(0.3)
    bar_val += 1
    out.update(progress(bar_val,len(interactors_uniprot_lst)))

204  Q08561
204  none


In [ ]:
#@title annotation dictionary
master_lst = []
domain_dict = {}
for uniprot_id in interactors_uniprot_lst:
  domain_dict[uniprot_id] = []
for uniprot_id in interactors_uniprot_lst:
    completeName = "./actin_dataset/interactor_representative_annotations_by_uniprot/"+uniprot_id+".json"
    with open(completeName, 'r') as openfile:
        result = json.load(openfile)
        #print(uniprot_id)
        if result != None:
          for domain in result['results']:
            for annot in domain['proteins']:
              prot_fragments = annot['entry_protein_locations']
              if prot_fragments != None:
                for fragment in prot_fragments:
                  for segment in fragment['fragments']:
                    rep_status = segment['representative']
                    frag_start = segment['start']
                    frag_end = segment['end']
                    entity_uniprot_id = annot['accession']
                    domain_name = domain['metadata']['name']
                    source = domain['metadata']['source_database']
                    domain_annot = domain['metadata']['accession']
                    annot_type = domain['metadata']['type']
                    domain_dict[uniprot_id].append({'accession': domain_annot, 'source': source, 'name': domain_name, 'type': annot_type, 'start':frag_start, 'end':frag_end, 'rep_status':rep_status})

In [ ]:
#@title Map representative domains to each actin interactor mod
master_lst = []

for key, value in domain_dict.items():
  for annot in value:
    master_lst.append([key, annot['accession'], annot['source'], annot['name'], annot['type'], annot['start'], annot['end'], annot['rep_status']])

df_domains = pd.DataFrame(master_lst, columns = ['uniprot_id', 'annot_id', 'annot_source', 'annot_name', 'annot_type', 'start', 'end', 'rep_status'])

df_domains_uniprots = list(df_domains['uniprot_id'])
df_domains_uniprots = list(set(df_domains_uniprots))

domain_annot_dict = {}
for unp in df_domains_uniprots:
  domain_annot_dict[unp] = []
#domain_annot_dict = defaultdict(list)

df_domains_dfhs = df_domains.query('(annot_type == "domain" & rep_status == True)|(annot_type == "domain" & annot_source == "interpro")|(annot_type == "family" & annot_source == "interpro")|(annot_type == "homologous_superfamily" & annot_source == "interpro")')
for index, row in df_domains_dfhs.iterrows():
  inner_dict = {'start':row['start'], 'end':row['end'], 'annot_id':row['annot_id'], 'annot_name':row['annot_name'], 'rep_status':row['rep_status'], 'annot_type':row['annot_type']}
  domain_annot_dict[row['uniprot_id']].append(inner_dict)

domain_annot_dict['P12883'] = [
    {
        'start':32,
        'end':81,
        'annot_id':'PS51844',
        'annot_name':'SH3_LIKE',
        'rep_status':True,
        'annot_type':'domain'
    },
    {
        'start':79,
        'end':779,
        'annot_id':'SM00242',
        'annot_name':'MYSc',
        'rep_status':True,
        'annot_type':'domain'
    },
    {
        'start':845,
        'end':1923,
        'annot_id':'IPR002928',
        'annot_name':'Myosin tail',
        'rep_status':False,
        'annot_type':'domain'
    }
                               ]

In [ ]:
remain_annot_unps = []
for k, v in domain_annot_dict.items():
  if v == []:
    remain_annot_unps.append(k)

In [ ]:
#@title check domain annotations
uniprot_id = 'Q8VQB5' #@param {type: "string"}
domain_annot_dict[uniprot_id]

KeyError: 'Q8VQB5'

In [ ]:
#@title load actin interactor interface dictionary
savepath = "./actin_dataset/actin_database_sheets"
with open(os.path.join(savepath,"partner_interface_dictionary_unp.pkl"), 'rb') as g:
    actin_interactor_interface_dict = pickle.load(g)

savepath = "./actin_dataset/actin_database_sheets"
with open(os.path.join(savepath,"interface_dictionary_msa.pkl"), 'rb') as g:
    actin_interactor_interface_dict_actin_res = pickle.load(g)

In [ ]:
#@title unavailable domain annotations
mod_actin_interactor_interface_dict = {}
mod_actin_interactor_interface_dict_actin_res = {}
for key1, value1 in actin_interactor_interface_dict.items():
  interactor_unp = key1[2][1]
  if interactor_unp in list(domain_annot_dict.keys()):
    mod_actin_interactor_interface_dict[key1] = value1
    mod_actin_interactor_interface_dict_actin_res[key1] = actin_interactor_interface_dict_actin_res[key1]
  else:
    print(key1, value1)

('8IAH_1', ('A_act', 'Q6QAQ1'), ('U', 'none')) []
('8IAH_1', ('I_act', 'Q6QAQ1'), ('U', 'none')) []
('3MN9_1', ('A_act', 'P10987'), ('X', 'none')) []
('7TPT_1', ('O_act', 'P68135'), ('h', 'none')) []
('7TPT_1', ('Q_act', 'P68135'), ('j', 'none')) []
('7TPT_1', ('P_act', 'P68135'), ('i', 'none')) []
('7TPT_1', ('N_act', 'P68135'), ('g', 'none')) []
('7TPT_1', ('S_act', 'P68135'), ('l', 'none')) []
('7TPT_1', ('R_act', 'P68135'), ('k', 'none')) []
('7TPT_1', ('I_act', 'P68135'), ('b', 'none')) []
('7TPT_1', ('H_act', 'P68135'), ('a', 'none')) []
('7TPT_1', ('S_act', 'P68135'), ('m', 'none')) []
('7TPT_1', ('H_act', 'P68135'), ('b', 'none')) []
('7TPT_1', ('R_act', 'P68135'), ('l', 'none')) []
('7TPT_1', ('I_act', 'P68135'), ('c', 'none')) []
('7TPT_1', ('Q_act', 'P68135'), ('k', 'none')) []
('7TPT_1', ('O_act', 'P68135'), ('i', 'none')) []
('7TPT_1', ('P_act', 'P68135'), ('j', 'none')) []
('7TPT_1', ('N_act', 'P68135'), ('h', 'none')) []
('7TPT_1', ('I_act', 'P68135'), ('d', 'none')) []


In [ ]:
#@title actin interactor domains (mod)
import numpy as np
import copy

actin_interactor_domains = defaultdict(list)

domain_annot_dict1 = copy.deepcopy(domain_annot_dict)
for key1, value1 in mod_actin_interactor_interface_dict.items():
  interactor_unp = key1[2][1]
  dom_len = 0
  domain_annotations = domain_annot_dict1[interactor_unp]
  for annot_idx, each_annot in enumerate(domain_annotations,start = 0):
    dom_start = each_annot['start']
    dom_end = each_annot['end']
    dom_status = each_annot['rep_status']
    int_res_count = 0
    for idx, int_res in enumerate(actin_interactor_interface_dict[key1], start = 1):
      if dom_start <= int_res <= dom_end:# and (dom_end - dom_start) >= dom_len:
        int_res_count+=1
    each_annot['int_res_count'] = int_res_count/ (dom_end - dom_start)
    if each_annot['int_res_count'] != 0: #and each_annot['annot_type'] == 'domain':
      actin_interactor_domains[key1].append(each_annot)

In [ ]:
actin_interactor_domains_cp = copy.deepcopy(actin_interactor_domains)

for key1, value1 in actin_interactor_domains_cp.items():
  int_domain_lst = [annotation for annotation in value1 if annotation['annot_type'] == "domain"]
  int_domain_rep_lst = [annotation['rep_status'] for annotation in value1]
  if len(int_domain_lst) == 1:
    actin_interactor_domains[key1] = [int_domain_lst[0]['annot_name']]
  elif len(int_domain_lst) > 1 and int_domain_rep_lst.count(True) == 1:
    for annotation in int_domain_lst:
      if annotation['rep_status'] == True:
        actin_interactor_domains[key1] = [annotation['annot_name']]
  elif len(int_domain_lst) > 1 and int_domain_rep_lst.count(True) > 1:
    int_res_count_lst_rep = [annotation['int_res_count'] for annotation in int_domain_lst if annotation['rep_status'] == True]
    for annotation in int_domain_lst:
      if annotation['rep_status'] == True and annotation['int_res_count'] == np.max(int_res_count_lst_rep):
        actin_interactor_domains[key1] = [annotation['annot_name']]
  elif len(int_domain_lst) >1 and int_domain_rep_lst.count(True) == 0:
    int_res_count_lst = [annotation['int_res_count'] for annotation in int_domain_lst]
    for annotation in int_domain_lst:
      if annotation['int_res_count'] == np.max(int_res_count_lst):
        actin_interactor_domains[key1] = [annotation['annot_name']]
  else:
    actin_interactor_domains[key1] = ['no_domain_annotation'] #864

for k,v in actin_interactor_domains.items():
  if len(v) > 1:
    print('domains>1', k)
    actin_interactor_domains[k] = [v[0]]

for key1, value1 in actin_interactor_domains_cp.items():
  int_family_lst = [annotation for annotation in value1 if annotation['annot_type'] == "family"]
  if len(int_family_lst) == 1:
    actin_interactor_domains[key1].append(int_family_lst[0]['annot_name'])
  elif len(int_family_lst) > 1:
    int_res_count_lst = [annotation['int_res_count'] for annotation in int_family_lst]
    for annotation in int_family_lst:
      if annotation['int_res_count'] == np.max(int_res_count_lst):
        actin_interactor_domains[key1].append(annotation['annot_name'])
  else:
    actin_interactor_domains[key1] .append('no_family_annotation')

for k,v in actin_interactor_domains.items():
  if len(v) > 2:
    print('families>1', k)
    actin_interactor_domains[k] = [v[0], v[1]]

for key1, value1 in actin_interactor_domains_cp.items():
  int_supfamily_lst = [annotation for annotation in value1 if annotation['annot_type'] == "homologous_superfamily"]
  if len(int_supfamily_lst) == 1:
    actin_interactor_domains[key1].append(int_supfamily_lst[0]['annot_name'])
  elif len(int_supfamily_lst) > 1:
    int_res_count_lst = [annotation['int_res_count'] for annotation in int_supfamily_lst]
    for annotation in int_supfamily_lst:
      if annotation['int_res_count'] == np.max(int_res_count_lst):
        actin_interactor_domains[key1].append(annotation['annot_name'])
  else:
    actin_interactor_domains[key1].append('no_supfamily_annotation')

for k,v in actin_interactor_domains.items():
  if len(v) > 3:
    print('supfamilies>1', k)
    actin_interactor_domains[k] = [v[0], v[1], v[2]]

families>1 ('1HLU_1', ('A_act', 'P60712'), ('P', 'P02584'))
families>1 ('3UB5_1', ('A_act', 'P60712'), ('P', 'P02584'))
families>1 ('3U4L_1', ('A_act', 'P60712'), ('P', 'P02584'))
families>1 ('2BTF_1', ('A_act', 'P60712'), ('P', 'P02584'))
supfamilies>1 ('1H1V_1', ('A_act', 'P68135'), ('G', 'P06396'))


In [ ]:
#@title check which key is missing
for key1, value1 in mod_actin_interactor_interface_dict.items():
  if key1 not in list(actin_interactor_domains.keys()):
    print(key1)
    pass
  # elif type(actin_interactor_domains[key1]) == list:
  #   print(key1)

('8UEE_1', ('K_act', 'P68135'), ('B', 'P0CL52'))
('8UEE_1', ('F_act', 'P68135'), ('C', 'P0CL52'))
('8UEE_1', ('I_act', 'P68135'), ('A', 'P0CL52'))
('7AD9_1', ('B_act', 'P68135'), ('A', 'Q08641'))
('7AD9_1', ('D_act', 'P68135'), ('C', 'Q08641'))
('7AD9_1', ('I_act', 'P68135'), ('G', 'Q08641'))
('7AD9_1', ('H_act', 'P68135'), ('L', 'Q08641'))
('7AD9_1', ('F_act', 'P68135'), ('E', 'Q08641'))
('7AD9_1', ('H_act', 'P68135'), ('G', 'Q08641'))
('7AD9_1', ('B_act', 'P68135'), ('L', 'Q08641'))
('7AD9_1', ('D_act', 'P68135'), ('E', 'Q08641'))
('7ZVW_1', ('B_act', 'Q9P4D1'), ('H', 'C4R7G0'))
('7Z7H_1', ('D_act', 'P68135'), ('F', 'Q8GF97'))
('7Z7H_1', ('B_act', 'P68135'), ('F', 'Q8GF97'))
('8DD0_1', ('C_act', 'P68137'), ('J', 'P42639'))
('8DD0_1', ('D_act', 'P68137'), ('O', 'P42639'))
('8C4E_1', ('C_act', 'P68139'), ('K', 'P0CL52'))
('8C4E_1', ('D_act', 'P68139'), ('L', 'P0CL52'))


In [ ]:
#@title save actin interactor domains
savepath = "./actin_dataset/actin_database_sheets"
with open(os.path.join(savepath,"actin_interactor_domains.pkl"), 'wb') as g:
    pickle.dump(actin_interactor_domains,g)

# Post processing interface matrix

In [ ]:
#@title load data
savepath = "./actin_dataset/actin_database_sheets"
with open(os.path.join(savepath,"actin_interface_matrix.pkl"), 'rb') as g:
    actin_interactor_interface_dict = pickle.load(g)

with open(os.path.join(savepath,"actin_interactor_domains.pkl"), 'rb') as g:
    actin_interactor_domains = pickle.load(g)

In [ ]:
#@title unavailable annotations
mod_actin_interactor_interface_dict = {}
for key1, value1 in actin_interactor_interface_dict.items():
  if key1 not in list(actin_interactor_domains.keys()):
    print(key1)
  else:
    mod_actin_interactor_interface_dict[key1] = value1

('8IAH_1', ('A_act', 'Q6QAQ1'), ('U', 'none'))
('8IAH_1', ('I_act', 'Q6QAQ1'), ('U', 'none'))
('3MN9_1', ('A_act', 'P10987'), ('X', 'none'))
('7TPT_1', ('O_act', 'P68135'), ('h', 'none'))
('7TPT_1', ('Q_act', 'P68135'), ('j', 'none'))
('7TPT_1', ('P_act', 'P68135'), ('i', 'none'))
('7TPT_1', ('N_act', 'P68135'), ('g', 'none'))
('7TPT_1', ('S_act', 'P68135'), ('l', 'none'))
('7TPT_1', ('R_act', 'P68135'), ('k', 'none'))
('7TPT_1', ('I_act', 'P68135'), ('b', 'none'))
('7TPT_1', ('H_act', 'P68135'), ('a', 'none'))
('7TPT_1', ('S_act', 'P68135'), ('m', 'none'))
('7TPT_1', ('H_act', 'P68135'), ('b', 'none'))
('7TPT_1', ('R_act', 'P68135'), ('l', 'none'))
('7TPT_1', ('I_act', 'P68135'), ('c', 'none'))
('7TPT_1', ('Q_act', 'P68135'), ('k', 'none'))
('7TPT_1', ('O_act', 'P68135'), ('i', 'none'))
('7TPT_1', ('P_act', 'P68135'), ('j', 'none'))
('7TPT_1', ('N_act', 'P68135'), ('h', 'none'))
('7TPT_1', ('I_act', 'P68135'), ('d', 'none'))
('7TPT_1', ('S_act', 'P68135'), ('n', 'none'))
('7TPT_1', ('

### interface matrix

In [ ]:
#@title interface matrix - all

mod_interface_dict = {}
for key, value in mod_actin_interactor_interface_dict_actin_res.items():
  # if 'act' in key[2][0]: ########this is changed
    mod_interface_dict[key] = value

heatmap_dict = {}
for chain_id, res_lst in mod_interface_dict.items():
  for res_idx, residue in consensus_dict.items():
    heatmap_dict[(chain_id,res_idx)] = 0.0
  for res in res_lst:
    heatmap_dict[(chain_id,res)] = 1.0

ser = pd.Series(list(heatmap_dict.values()),index=pd.MultiIndex.from_tuples(heatmap_dict.keys()))
interface_df = ser.unstack().fillna(0)

savepath = "./actin_dataset/actin_database_sheets" #param {type: "string"}
interface_df.to_csv(os.path.join(savepath,"actin_interface_matrix_all.csv"))


In [ ]:
#@title domain annotations df
dom_lst_len = 0
for key, value in actin_interactor_domains.items():
  dom_lst = value
  if len(dom_lst) > dom_lst_len:
    dom_lst_len = len(dom_lst)

for key, value in actin_interactor_domains.items():
  dom_lst = value
  actin_interactor_domains[key] = dom_lst

dom_dict = {}
for key, value in actin_interactor_domains.items():
  for domidx, domtype in enumerate(['domain', 'family', 'superfamily'], start = 1):
    dom_dict[(key, domtype)] = value[domidx-1]

ser = pd.Series(list(dom_dict.values()),index=pd.MultiIndex.from_tuples(dom_dict.keys()))
dom_df = ser.unstack().fillna(0)

savepath = "./actin_dataset/actin_database_sheets" #param {type: "string"}
dom_df.to_csv(os.path.join(savepath,"actin_interactor_domain_annot_temp.csv"))